In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#파일 업로드
import shutil
try:
  shutil.copy('/content/gdrive/MyDrive/text/11주차/data.zip', '/content/')
  print('dataset download!!!')
except Exception as err:
  print(str(err))

dataset download!!!


In [3]:
# 압축 풀고 디렉토리 생성

import zipfile

with zipfile.ZipFile('/content/data.zip', 'r') as target_file:
    target_file.extractall('/content/data/')
print('ok')

ok


In [4]:
!pip install ujson
!pip install konlpy 
!pip install pyLDAvis 
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 51.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 8.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=1397528c23ea9fc4059ccf703f613cb3906e5bad8e322b5226090ac2eb04e1f1
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
Looking in 

In [5]:
# Data Preprocessing Package
import re                       #정규식
import numpy as np
import pandas as pd
import os                      #디렉토리와 경로정보

# NLP Package
from konlpy.tag import * 
import gensim                    #토픽모델링을 하는 라이브러리
import gensim.corpora as corpora #텍스트분석
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package   #LDA시각화
import pyLDAvis 
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


from pprint import pprint #pretty print
import itertools #iterable 객체 처리
import math

import logging #로그처리
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings #경고 무시
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [6]:
# 기본 저장 주소
ROOT_PATH ="./data/"

# 데이터 저장 주소
DATA_FOLDER_PATH = os.path.join(ROOT_PATH,"1.Data_my/")
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"윤석열_f.csv")                  # Portal News Crawler에서 저장한 데이터 
DATA_STOP_WORD_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"Data_stop_word.txt")   # 불용어 사전 

# 모델 저장 주소
MODEL_SAVE_FOLDER = os.path.join(ROOT_PATH,"2.Model_my/")
LDA_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_lda_topic_modeling.lda")
OPT_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_opt_topic_modeling.lda")

# 결과물 저장 주소
RESULT_FOLDER = os.path.join(ROOT_PATH,"3.Result_my/")
RESULT_SAVE_LDAVIS = os.path.join(RESULT_FOLDER,"Result_lda_vis.html")
RESULT_TOPIC_EXCEL = os.path.join(RESULT_FOLDER,"Result_topic_excel.xlsx")
RESULT_corpus_EXCEL = os.path.join(RESULT_FOLDER,"Result_corpus_excel.xlsx")

In [7]:
# 데이터 불러오기 
DF_raw = pd.read_csv(DATA_FILE_NAME) # 엑셀파일의 sheet1시트의 텍스트 가져오기
len(DF_raw)

# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',DF_raw.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
DF_raw = DF_raw.dropna(how = 'any') # Null 값이 존재하는 행 제거
DF_raw = DF_raw.drop_duplicates()   # 중복 데이터 프레임 제거 
DF_raw = DF_raw.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', DF_raw.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
print("처리할 데이터수 : ",len(DF_raw))


Null값이 있는지 확인합니다. False

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False

처리할 데이터수 :  3947


In [8]:
DF_raw

윤_기사작성일                                          윤_기사제목  \
0     2022.03.07.           [사설]사전투표 1632만… ‘비호감 대선’ 물밑에서 분출하는 민심   
1     2022.03.07.  [대선 1.5℃⑥] '청기행' 4년의 결론..."새 정부 최우선 과제는 기후 위기"   
2     2022.03.07.                       [취재석] '또 철수'…안철수의 아리송한 행보   
3     2022.03.07.                역대급 사전투표율…여 “지지층 결집” 야 “정권교체 열망”   
4     2022.03.07.                윤석열, 경기 8개 지역 순환 유세...안철수와 합동 유세   
...           ...                                             ...   
3942  2022.03.08.        [박성진의 한국군 코멘터리] 사드보다 ‘K미사일’이 호갱에서 벗어나는 길   
3943  2022.03.08.               윤석열 "강직·겸허하게 나아갈 것…제 소신에 희망 걸어달라"   
3944  2022.03.08.           [사설] 유세 중 벌어진 대낮 망치 테러, 대선 후보 경호 강화해야   
3945  2022.03.08.              국민의힘, 與 '김만배 녹취록' 공세에 '제2의 생태탕' 역공   
3946  2022.03.08.                 中전문가 "대선 누가 돼도 중국 영향력 무시 못 할 것"   

                                                 윤_기사주소  
0     https://www.donga.com/news/article/all/2022030...  
1          http://news.tf.co.kr/read/ptoday/1922833.htm  
2          http://news.tf.co.kr/read/ptoday/1922777.htm  
3           https://www.joongang.co.kr/article/25053334  
4     https://www.ytn.co.kr/_ln/0101_202203070002360435  
...                                                 ...  
3942  https://www.khan.co.kr/opinion/column/article/...  
3943  https://www.dailian.co.kr/news/view/1090968/?s...  
3944  https://www.chosun.com/opinion/editorial/2022/...  
3945  https://www.dailian.co.kr/news/view/1090967/?s...  
3946    http://www.newspim.com/news/view/20220307000909  

[3947 rows x 3 columns]

In [9]:
# raw데이터로부터 텍스트만 불러오기 
DF_only_text = DF_raw['윤_기사제목'] #엑셀필드명

LDA

In [10]:
tokenizer = Okt() # 토큰나이저 지정
stopword_vocab = DATA_STOP_WORD_FILE_NAME # 불용어 파일 불러오기
sep = "\n" # 불용어 처리 인자

def build_vocab(data_frame ,stopword_vocab, separate):
    
    # 불용어 데이터를 가져와 리스트로 변환합니다.
    with open(stopword_vocab, encoding = 'utf-8') as f:
        temp1 = []
        for i in f:
            temp1.append(i)
            
    globals()['stopword_vocab'] = []
    
    # 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
    # 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
    for j in range(len(temp1)):
        temp2 = temp1[j].rstrip(separate)
        globals()['stopword_vocab'].append(temp2)
    
    #okt token에서 명사만 출력합니다. 단어의 길이가 1 초과인 단어만 출력합니다. 
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.nouns(data_frame[i])
        for j in range(len(temp)):

            if len(temp[j]) > 1:
                num_list.append(temp[j])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data =build_vocab(DF_only_text, stopword_vocab, sep)


# 전체 에 대한 워드 카운트 계수 확인

def word_corpus(result_data):
    #전체 단어의 갯수 파악
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    #단어의 빈도수를 확인 후 추가할 불용어 확인 작업
    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 n개의 단어만 저장 vocab
    return vocab

vocab=word_corpus(result_data)

# 전체 워드의 빈도 계수 
df_corpus=pd.DataFrame(columns=["text","count"])
tmp_list=[]
tmp_list1=[]
for word, num in vocab:
    tmp_list.append(word)
    tmp_list1.append(num)
df_corpus['text']=tmp_list
df_corpus['count']=tmp_list1
#상위 20개의 워드 카운드 계수만 출력
a=df_corpus.head(1000)
print(df_corpus.head(20))

#빈도분석한 결과를 별도의 파일에 저장함
a.to_excel(RESULT_corpus_EXCEL ,sheet_name = "sheet1")


# 토픽 모델링 딕셔너리 생성
id2word = corpora.Dictionary(result_data)
 
# 토픽모델링에 사용할 말뭉치 생성
texts = result_data
 
# 용어-문서 빈도
corpus = [id2word.doc2bow(text) for text in texts]


전체 워드의 개수 : 22458
    text  count
0     대선    814
1     후보    767
2     유세    544
3     지지    407
4     만배    371
5     호소    324
6    안철수    315
7     국민    268
8     포토    249
9    녹취록    243
10   대장동    189
11   민주당    183
12   송영길    175
13    투표    174
14    선거    150
15   어퍼컷    149
16    막판    148
17    부산    148
18    제주    140
19  사전투표    139


In [11]:
# Perplexity와 Coherence Score 을 판단
#print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
#print(' ')

#NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
#TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
#save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

TOPICS_W_NUM =20
save_lda_model=0
RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')
for i in range(1,30):
  NUM_TOPICS=i
 
  #해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

  # 토픽 출력
 #  pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
  doc_lda = lda_model[corpus]

  # 모델 저장 
  if save_lda_model == 0:
      lda_model.save(LDA_MODEL_SAVE_NAME)
  # 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)
  """
  해당 셀은 설계한 모델을 계산하는 셀입니다.
  측정은 Perplexity와 Coherence Score입니다.
  """
  #print('\nNUM_TOPICS',NUM_TOPICS)
  # Perplexity 
  #print('Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
  
  # Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  #print('Coherence Score: ', coherence_lda)
 # print('NUM_TOPICS',NUM_TOPICS,'Perplexity: ', lda_model.log_perplexity(corpus),'Coherence: ', coherence_lda)

  print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)


NUM_TOPICS perplexity coherence
T 1 -6.986427790229621 0.42058860348883487
T 2 -6.943051989434536 0.3968088058950867
T 3 -7.047983942144843 0.4246834228240977
T 4 -7.185519679057777 0.45903961955234973
T 5 -7.195047461479751 0.4576787459342704
T 6 -7.2983316714563635 0.5218020477686968
T 7 -7.348792856043111 0.5614729892051946
T 8 -7.363426979774836 0.5288143674018647
T 9 -7.416833063096713 0.5806587109367953
T 10 -7.421415768945267 0.5470958685267096
T 11 -7.486045914308738 0.5779182835253202
T 12 -7.483859162336095 0.5696652578209426
T 13 -7.510303412350365 0.5844035649781765
T 14 -7.519962643116503 0.5870084256249772
T 15 -7.5016700263845175 0.5768759827810943
T 16 -7.530295248350474 0.5742719538825405
T 17 -7.574001708542614 0.5764463499089647
T 18 -7.549913027038057 0.5638101506886845
T 19 -7.547641749938414 0.5695680905197814
T 20 -7.5746198833646945 0.5778615095161961
T 21 -7.592325933144241 0.5734219531972042
T 22 -7.6258467605404014 0.5854502162377812
T 23 -7.637477892407943 0

In [12]:
print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
print(' ')

NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True

#해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                            num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                            update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                            passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

# 토픽 출력
pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
doc_lda = lda_model[corpus]

# 모델 저장 
if save_lda_model == 0:
    lda_model.save(LDA_MODEL_SAVE_NAME)
# 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)

토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.
 
토픽의 개수를 입력해 주세요. 10
출력할 토픽별 단어의 개수를 입력해 주세요 10
선택한 토픽 모델을 저장하시겠습니까? 
0 저장  
1 미저장  0
[(0,
  '0.068*"녹취록" + 0.057*"만배" + 0.053*"이준석" + 0.042*"국힘" + 0.026*"선택" + '
  '0.022*"달라" + 0.022*"조작" + 0.020*"공작" + 0.016*"생태" + 0.014*"추천"'),
 (1,
  '0.228*"대선" + 0.199*"후보" + 0.046*"여야" + 0.044*"민주당" + 0.039*"평택" + '
  '0.024*"정책" + 0.017*"막판" + 0.013*"이틀" + 0.013*"만난" + 0.012*"부실"'),
 (2,
  '0.175*"유세" + 0.077*"안철수" + 0.058*"제주" + 0.034*"집중" + 0.028*"화성" + '
  '0.023*"국정" + 0.022*"공략" + 0.021*"마지막" + 0.020*"수도권" + 0.020*"부선"'),
 (3,
  '0.144*"지지" + 0.106*"국민" + 0.051*"공약" + 0.047*"경기" + 0.033*"선언" + 0.029*"방문" '
  '+ 0.020*"민주주의" + 0.020*"지역" + 0.019*"광장" + 0.016*"청년"'),
 (4,
  '0.102*"호소" + 0.079*"포토" + 0.058*"대장동" + 0.034*"만배" + 0.033*"부산" + '
  '0.026*"의혹" + 0.023*"수사" + 0.022*"공개" + 0.022*"몸통" + 0.021*"종합"'),
 (5,
  '0.037*"경제" + 0.022*"언론" + 0.022*"규탄" + 0.022*"시민" + 0.019*"선거운동" + '
  '0.017*"극복" + 0.015*"검찰" + 0.015*"특검" + 0.014*"김건희" + 0.

In [13]:
#토픽평가
"""
해당 셀은 설계한 모델을 계산하는 셀입니다.
측정은 Perplexity와 Coherence Score입니다.
"""

# Perplexity 
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
# Perplexity는 작을 수록 Coherence Score는 높을 수록 좋다.모델 1개의 값
# 토픽의 개수를 다르게 하여 판단비교해보세요. 동시에 높아지니 믿어도 되겠다. 하나만 선택해라면 코히어런스를 써
# 젠심 코히러런스로 검색해봐서 coherence='c_v'값을 바꿔가면서 해보세요


Perplexity:  -7.421415768945267

Coherence Score:  0.5470958685267096


In [14]:
for topic_id in range(NUM_TOPICS):
    topic_word_probs = lda_model.show_topic(topic_id, TOPICS_W_NUM)
    print("Topic ID: {}".format(topic_id))

    for topic_word, prob in topic_word_probs:
        print("\t{}\t{}".format(topic_word, prob))
    print("\n")

Topic ID: 0
	녹취록	0.06765700876712799
	만배	0.05703399330377579
	이준석	0.05317606031894684
	국힘	0.04161771386861801
	선택	0.025631895288825035
	달라	0.022450242191553116
	조작	0.0216342955827713
	공작	0.020422952249646187
	생태	0.01637958735227585
	추천	0.014169433154165745


Topic ID: 1
	대선	0.22761739790439606
	후보	0.1993066370487213
	여야	0.045953501015901566
	민주당	0.04399803280830383
	평택	0.03929522633552551
	정책	0.024052012711763382
	막판	0.016545964404940605
	이틀	0.013227040879428387
	만난	0.012948411516845226
	부실	0.012247959151864052


Topic ID: 2
	유세	0.17528527975082397
	안철수	0.07704237848520279
	제주	0.057977236807346344
	집중	0.0337507389485836
	화성	0.027878019958734512
	국정	0.02322077378630638
	공략	0.022122591733932495
	마지막	0.020632417872548103
	수도권	0.019997112452983856
	부선	0.019783297553658485


Topic ID: 3
	지지	0.14401625096797943
	국민	0.10629476606845856
	공약	0.05104051157832146
	경기	0.046867113560438156
	선언	0.033292923122644424
	방문	0.029018091037869453
	민주주의	0.01995794102549553
	지역	0.019849739968776703
	광장	0.019

In [15]:
"""
위의 셀에서 학습한 모델을 시각화 하여 HTML 파일로 저장하는 셀입니다. 
문서의 양이 많을 경우 오래 걸리니 참고 바랍니다. 
"""
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    pyLDAvis.save_html(vis, RESULT_SAVE_LDAVIS)
    return vis
#lda_model or optimal_model
create_vis(lda_model)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.155881  0.012244       1        1   8.272435
1     -0.288789  0.198615       2        1  14.545199
2     -0.169064 -0.362631       3        1  13.246102
3      0.002134  0.000138       4        1  10.573579
4      0.193255 -0.020526       5        1  12.939863
5      0.043338  0.014356       6        1   7.764476
6      0.041370  0.014586       7        1   8.175770
7     -0.057799  0.092632       8        1   8.049408
8      0.032620  0.009967       9        1   8.021096
9      0.047053  0.040619      10        1   8.412073, topic_info=      Term        Freq       Total Category  logprob  loglift
0       대선  744.000000  744.000000  Default  30.0000  30.0000
46      후보  651.000000  651.000000  Default  29.0000  29.0000
26      유세  522.000000  522.000000  Default  28.0000  28.0000
194     지지  342.000000  342.000000  Default  27.0000  27.0000
195     호소  298.000000  298.000000  Default  26.0000  26.0000
...    ...         ...         ...      ...      ...      ...
1892    사채   12.038134   12.834473  Topic10  -5.0558   2.4114
2701   자신감   10.941157   11.737845  Topic10  -5.1514   2.4052
1440    준비   10.582604   11.378964  Topic10  -5.1847   2.4029
6     사전투표   49.357283   75.697745  Topic10  -3.6448   2.0478
77      투표   59.227226  103.688947  Topic10  -3.4625   1.9155

[341 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1383      9  0.954296   가짜
977       8  0.974270   가치
573       2  0.974364   강조
518       8  0.931558   개혁
1812      1  0.921332  거렁뱅
...     ...       ...  ...
195       5  0.999050   호소
859       3  0.991363   화성
914       1  0.915153   확인
46        2  0.998727   후보
2265     10  0.979389   희망

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [16]:
# 위에서 선택한 토픽 모델을 엑셀 파일로 떨어뜨리기 위한 함수 
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data):
    sent_topics_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        topics_info_by_doc = row[0]
        topics_info_by_doc = sorted(topics_info_by_doc, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(topics_info_by_doc):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]),
                                                       ignore_index=True)
            else:
                break

    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


'''
확인하고자 하는 모델을 입력하여 데이터 프레임으로 만든 후, 최종 엑셀파일을 저장합니다.
'''

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data)
topic_weight=lda_model[corpus]
df_topic_weight = pd.DataFrame()
for i in range(1, lda_model.num_topics+1):
    df_topic_weight['topic{}'.format(i)]=pd.Series()


# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
 

# LDA모델에서 토픽 웨이트를 추출하여 데이터 프레임에 저장 
for num, topic in enumerate(topic_weight):
    for i in topic[0]:
        df_topic_weight.loc[num, 'topic' + str(i[0]+1)] = i[1]
        #df_topic_weight.loc[num, 'topic' + str(i[0])] = i[1] 
# Null값은 아주 작은 값으로 대체
df_topic_weight = df_topic_weight.fillna(math.exp(-1000)) 

#데이터 프레임 연결
df_topic_last = pd.concat([DF_raw, df_dominant_topic, df_topic_weight], axis=1)
print(df_topic_last)

# 마지막 엑셀 저장
df_topic_last.to_excel(RESULT_TOPIC_EXCEL,sheet_name = "sheet1")

          윤_기사작성일                                          윤_기사제목  \
0     2022.03.07.           [사설]사전투표 1632만… ‘비호감 대선’ 물밑에서 분출하는 민심   
1     2022.03.07.  [대선 1.5℃⑥] '청기행' 4년의 결론..."새 정부 최우선 과제는 기후 위기"   
2     2022.03.07.                       [취재석] '또 철수'…안철수의 아리송한 행보   
3     2022.03.07.                역대급 사전투표율…여 “지지층 결집” 야 “정권교체 열망”   
4     2022.03.07.                윤석열, 경기 8개 지역 순환 유세...안철수와 합동 유세   
...           ...                                             ...   
3942  2022.03.08.        [박성진의 한국군 코멘터리] 사드보다 ‘K미사일’이 호갱에서 벗어나는 길   
3943  2022.03.08.               윤석열 "강직·겸허하게 나아갈 것…제 소신에 희망 걸어달라"   
3944  2022.03.08.           [사설] 유세 중 벌어진 대낮 망치 테러, 대선 후보 경호 강화해야   
3945  2022.03.08.              국민의힘, 與 '김만배 녹취록' 공세에 '제2의 생태탕' 역공   
3946  2022.03.08.                 中전문가 "대선 누가 돼도 중국 영향력 무시 못 할 것"   

                                                 윤_기사주소  Document_No  \
0     https://www.donga.com/news/article/all/2022030...            0   
1          http://news.tf.c

분석 결과 

: 토픽 개수 10개, 토픽별 단어 10개 -> 혼란도 -7.421415768945267, 일관성 0.5470958685267096


토픽1에서 가장 높은 수치를 가진 단어 : 녹취록
토픽2에서 가장 높은 수치를 가진 단어 : 대선
토픽3에서 가장 높은 수치를 가진 단어: 유세
토픽4에서 가장 높은 수치를 가진 단어: 지지
토픽5에서 가장 높은 수치를 가진 단어: 호소
토픽6에서 가장 높은 수치를 가진 단어: 경제
토픽7에서 가장 높은 수치를 가진 단어: 송영길
토픽8에서 가장 높은 수치를 가진 단어: 선거
토픽9에서 가장 높은 수치를 가진 단어: 머슴
토픽10에서 가장 높은 수치를 가진 단어: 민심

토픽 4,6,7,9,10 관련성 높음  : 이 토픽들은 토픽 단어들을 보아 당시 윤석열 후보의 공약과 유세, 지지에 관련된 토픽임을 추측할 수 있다

1번과 5번 토픽도 관련도 어느정도 높다고 볼 수 있음 : 이 토픽들은 당시 김만배, 녹취록 사건 등과 관련된 기사의 토픽임을 추측할 수 있다 
2번과 3번 토픽은 동떨어져있는 것으로 보아 다른 토픽들과 관련성이 떨어진다: 2번 토픽은 당시 지방유세와 선거와 관련되어있는 토픽이며 3번 토픽은 지역별 유세와 안철수와 관련된 기사 토픽임을 알 수 있다.  

LSA

In [17]:
DF_only_text

0                [사설]사전투표 1632만… ‘비호감 대선’ 물밑에서 분출하는 민심
1       [대선 1.5℃⑥] '청기행' 4년의 결론..."새 정부 최우선 과제는 기후 위기"
2                            [취재석] '또 철수'…안철수의 아리송한 행보
3                     역대급 사전투표율…여 “지지층 결집” 야 “정권교체 열망”
4                     윤석열, 경기 8개 지역 순환 유세...안철수와 합동 유세
                             ...                      
3942          [박성진의 한국군 코멘터리] 사드보다 ‘K미사일’이 호갱에서 벗어나는 길
3943                 윤석열 "강직·겸허하게 나아갈 것…제 소신에 희망 걸어달라"
3944             [사설] 유세 중 벌어진 대낮 망치 테러, 대선 후보 경호 강화해야
3945                국민의힘, 與 '김만배 녹취록' 공세에 '제2의 생태탕' 역공
3946                   中전문가 "대선 누가 돼도 중국 영향력 무시 못 할 것"
Name: 윤_기사제목, Length: 3947, dtype: object

In [18]:
from gensim import models

In [19]:
# ★ 모델링 후 각 토픽별로 중요한 단어들을 표시
NUM_TOPIC_WORDS=TOPICS_W_NUM
def print_topic_words(model):
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))

        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
        print("\n")

In [20]:
model_LSA = models.lsimodel.LsiModel(corpus, num_topics=NUM_TOPICS,id2word=id2word)

In [21]:
print_topic_words(model_LSA)    

Topic ID: 0
	대선	0.6068687768565904
	후보	0.6040719450570349
	유세	0.27571924050200175
	지지	0.24248738481304283
	호소	0.20987895091029965
	포토	0.11327909325259254
	안철수	0.10623988478804591
	국민	0.10061547975071052
	어퍼컷	0.05960162077754175
	경기	0.05107460906553191


Topic ID: 1
	만배	0.6401288803279134
	녹취록	0.44774331108643445
	지지	-0.23033234216763424
	대장동	0.19221133007417573
	호소	-0.19005774415836385
	대선	0.1593457710460819
	통해	0.14749356938354863
	후보	-0.13960354075921613
	사건	0.13604367189578284
	부산	0.12321570789162523


Topic ID: 2
	유세	-0.6010494432117308
	지지	0.5104608312551268
	호소	0.4171156989725688
	만배	0.20959709898311857
	안철수	-0.17383720779569817
	녹취록	0.1464789797519228
	대선	-0.1442527538604008
	후보	0.1073265181577238
	나선	-0.07937651266705573
	집중	-0.07267096803781325


Topic ID: 3
	대선	-0.5964530669259506
	유세	0.5076539789822884
	안철수	0.2906810570004889
	지지	0.2757399619580319
	포토	0.21808877291312626
	호소	0.217589649495401
	만배	0.16731529081309948
	녹취록	0.1132060307462261
	경기	0.09140103639767351
	하남	0.0779

LSA로 모델링한 결과 토픽 1-10까지 
LSA로 토픽모델링 한 결과 
'대선, 만배, 지지, 유세, 대선, 대선, 안철수, 투표, 국민, 녹취록'이 높은 수치 기록 